In [ ]:
pip install selenium

In [3]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import cgi
import shutil
import os
import json
from selenium import webdriver

seperator = ', '
def getBandTree(band, page):
    if type(page) == int:
        page = str(page)
    theURL = 'https://www.ultimate-guitar.com/search.php?band_name=' + band + \
        '&type%5B4%5D=500&rating%5B4%5D=5&approved%5B1%5D=1&page=' + page + \
        '&view_state=advanced&tab_type_group=text&app_name=ugt&order=myweight'
 
    pageBand = requests.get(theURL)
    return BeautifulSoup(pageBand.content)
 
def download(browser, id, i = 0):
    #browser = webdriver.Chrome()
    browser.get("https://tabs.ultimate-guitar.com/download/public/" + id)
    time.sleep(1)
       
    if i == 0:    
        #click info
        doc = browser.find_elements_by_xpath('/html/body/div[3]/div/div/div/div/footer/button[1]')[0]
        doc.click()
    
    doc = browser.find_elements_by_xpath('/html/body/div[1]/div/div/div[2]/main/div[2]/div[2]/article/div[1]/section[2]/form/div/button')[0]
    doc.click()
    #browser.quit()  

    
def main():
    band = 'lamb of god'
    page = 1
    bandURL = getBandTree(band, page)
    ids = []
    #Get max pages
    pages = bandURL.find_all('div', { "class" : "paging" })
    maxPages = str(pages).count('')
    dfs = []
    for page in range(1, maxPages + 1):
        soup = getBandTree(band, page)
        #print(bandURL)
        #mybs = soup.find_all('script')
        #print(mybs[7])
        pattern = re.compile(r"window.UGAPP.store.page\s+=\s+(\{.*?\});\n")
        script = soup.find("script", text=pattern)

        data = pattern.search(script.text).group(1)
        data = json.loads(data)
        for row in data["data"]["results"]: #type = Pro
            #if 'ktype' in row.keys():
            pos = row["tab_url"].find("guitar_pro_")
            if pos > 0:
                id = row["tab_url"][pos:].replace('guitar_pro_', '')
                ids.append(id)
                #download(id)
    

    browser = webdriver.Chrome()
    
    i = 0
    for id in ids:
        print(id)
        download(browser, id, i)
        i = i + 1
    browser.quit()
        
if __name__=='__main__':
    main()
    
#/artist/slipknot_505

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import cgi
import shutil
import os
import json
import string
import csv
import time
import random 

def getBandList(letter, page):
    theURL = 'https://www.ultimate-guitar.com/bands/' + letter + str(page) +'.htm'
    pageBand = requests.get(theURL)
    return BeautifulSoup(pageBand.content)

def getList(file):
    letters = list(string.ascii_lowercase)
    letters .append('0-9')
    random.shuffle(letters)
    print(letters)
    page = 1
    results = []
    for letter in letters:
        time.sleep(30)
        time.sleep(random.randint(0, 25) / 3)
        print("Proccesing: " + letter)
        for page in range(1, 200):   
            time.sleep(12)
            time.sleep(random.randint(0, 6) / 3)
            print("page " + str(page))
            soup = getBandList(letter, page)
            pattern = re.compile(r"window.UGAPP.store.page\s+=\s+(\{.*?\});\n")
            script = soup.find("script", text=pattern)
            data = pattern.search(script.text).group(1)
            data = json.loads(data)
            if data:
                for row in data["data"]["artists"]: 
                    for k,v in row.items():
                        row.update({k:str(v)})
                    #results.append(row)
                    file.write(', '.join(row.values()) + '\n')
            else:
                break
    return results

output = open('output/band_list.csv', 'w+') 
headers = ['id', 'name', 'tabscount', 'artisct_url', 'tabs_last_update_timestamp']
output.write(', '.join(headers) + '\n')
#pasing files
datadir = 'data/'
data = getList(output)
output.close()
    


    
#https://www.ultimate-guitar.com/bands/a.htm

In [4]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import cgi
import shutil
import os
import json
import time
from selenium import webdriver
from pathlib import Path
import random 



seperator = ', '
def getBandTree(band, page):
    if type(page) == int:
        page = str(page)
    theURL = 'https://www.ultimate-guitar.com/search.php?band_name=' + band + \
        '&type%5B4%5D=500&rating%4B4%5D=5&approved%5B1%5D=1&page=' + page + \
        '&tab_type_group=text&app_name=ugt&order=myweight'
 
    pageBand = requests.get(theURL)
    return BeautifulSoup(pageBand.content)
 
def download(browser, id, i = 0):
    browser.get("https://tabs.ultimate-guitar.com/download/public/" + id)
    time.sleep(3)
       
    if i == 0:    
        # click popup
        time.sleep(2)
        doc = browser.find_elements_by_xpath('/html/body/div[3]/div/div/div/div/footer/button[1]')[0]
        doc.click()
    
    time.sleep(random.randint(0, 15) / 3)
    doc = browser.find_elements_by_xpath('/html/body/div[1]/div/div/div[2]/main/div[2]/div[2]/article/div[1]/section[2]/form/div/button')[0]
    doc.click()

    
def get_chrome_driver(download_dir):
    chrome_options = webdriver.ChromeOptions()
    prefs = {
      "download.default_directory": download_dir,
      "download.prompt_for_download": False,
      "download.directory_upgrade": True,
      "safebrowsing.enabled": True
    }
    chrome_options.add_experimental_option("prefs", prefs)

    return webdriver.Chrome(options=chrome_options)    
    
def main():
    band = 'lamb of god'
    curr_dir = Path.cwd()
    download_dir = str(curr_dir) + "/data/" + band
    
    page = 1
    bandURL = getBandTree(band, page)
    ids = []
    #Get max pages
    pages = bandURL.find_all('div', { "class" : "paging" })
    maxPages = str(pages).count('')
    print('max' + str(maxPages))
    dfs = []
    print('parsing..')
    for page in range(1, maxPages + 1):
        soup = getBandTree(band, page)
        pattern = re.compile(r"window.UGAPP.store.page\s+=\s+(\{.*?\});\n")
        script = soup.find("script", text=pattern)

        data = pattern.search(script.text).group(1)
        data = json.loads(data)
        for row in data["data"]["results"]: #type = Pro
            #if 'ktype' in row.keys():
            pos = row["tab_url"].find("guitar-pro-")
            if pos > 0:
                id = row["tab_url"][pos:].replace('guitar-pro-', '')
                ids.append(id)
                #download(id)

    browser = get_chrome_driver(download_dir)    
    
    print('downloading..')
    i = 0
    for id in ids:
        print(id)
        download(browser, id, i)
        i = i + 1
    #browser.quit()
        
if __name__=='__main__':
    main()
    
#https://www.ultimate-guitar.com/artist/exodus_4988?filter=guitar_pro

max3
parsing..
downloading..
223656
223657
449507
578934
608139
902860
1025393
1030591
1105246
1410663
1743492
1744507
1745625
1746107
223659
529250
816639
394240
396979
422612
484459
540544
722740
895964
1035390
413494
415286
485567
522469
564540
733455
908597
915917
2449747
1130936
1740343
1116724
2174985
223653
223654
223655
439621
440438
441939
443713
466233
484447
587213
816636
1743260
243850
376627
447372
498528
506139
816645
1090219
416802
462315
646718
1035387
223645
223646
440847
450938
510183
520493
940130
1116247
223664
412989
621433
895648
783939
786286
787281
807691
893389
415139
437253
437512


IndexError: list index out of range